In [1]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
import numpy as np 

In [2]:
# READ-IN the dataset
data = pd.read_csv('train.csv', index_col=2)
data.head()

,Item_ID,Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
Item_Store_ID,,,,,,,,,,,,
DRA12_BABATUNJI010,DRA12,BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
DRA12_BABATUNJI013,DRA12,BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
DRA12_BABATUNJI017,DRA12,BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
DRA12_BABATUNJI018,DRA12,BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
DRA12_BABATUNJI035,DRA12,BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77


In [3]:
# data.info()

In [4]:
# data.describe()

In [5]:
data.isnull().sum()

Item_ID                   0
Store_ID                  0
Item_Weight             802
Item_Sugar_Content        0
Item_Visibility           0
Item_Type                 0
Item_Price                0
Store_Start_Year          0
Store_Size             1450
Store_Location_Type       0
Store_Type                0
Item_Store_Returns        0
dtype: int64

In [6]:
data

,Item_ID,Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Returns
Item_Store_ID,,,,,,,,,,,,
DRA12_BABATUNJI010,DRA12,BABATUNJI010,11.60,Low Sugar,0.068535,Soft Drinks,357.54,2005,NaN,Cluster 3,Grocery Store,709.08
DRA12_BABATUNJI013,DRA12,BABATUNJI013,11.60,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,Supermarket Type1,6381.69
DRA12_BABATUNJI017,DRA12,BABATUNJI017,11.60,Low Sugar,0.041178,Soft Drinks,350.79,2014,NaN,Cluster 2,Supermarket Type1,6381.69
DRA12_BABATUNJI018,DRA12,BABATUNJI018,11.60,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,Supermarket Type2,2127.23
DRA12_BABATUNJI035,DRA12,BABATUNJI035,11.60,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,Supermarket Type1,2481.77
...,...,...,...,...,...,...,...,...,...,...,...,...
NCZ54_BABATUNJI019,NCZ54,BABATUNJI019,NaN,Low Sugar,0.145952,Household,402.39,1992,Small,Cluster 1,Grocery Store,406.14
NCZ54_BABATUNJI027,NCZ54,BABATUNJI027,NaN,Low Sugar,0.082956,Household,410.14,1992,Medium,Cluster 3,Supermarket Type3,13808.69
NCZ54_BABATUNJI045,NCZ54,BABATUNJI045,14.65,Low Sugar,0.083528,Household,406.14,2009,NaN,Cluster 2,Supermarket Type1,5685.93


In [7]:

def create_one_hot(dataset, column, prefix):
    encoded_column = pd.get_dummies(dataset[column], prefix=prefix)
    dataset = pd.concat([dataset, encoded_column], axis=1)
    dataset.drop(columns=column, inplace=True)
    return dataset

def prepare_dataset(dataset):
    dataset.drop(columns=['Store_ID', 'Item_ID',  'Item_Sugar_Content', 'Item_Visibility', 'Item_Weight', 'Store_Size'], inplace=True)
    
    # Apply feature transformation to Item Prices.
    dataset['Log_Item_Price'] = np.log(dataset['Item_Price'])
    dataset['Sqrt_Item_Price'] = np.sqrt(dataset['Item_Price'])
    
    dataset['Reciprocal_Item_Price'] = np.reciprocal(dataset['Item_Price'])
    
    dataset['Item_Price_Inv'] = 1 / (dataset['Item_Price'] + 1e-5)
    
    dataset['Cubert_Item_Price'] = np.cbrt(dataset['Item_Price'])
    
    dataset = create_one_hot(dataset, 'Item_Type', 'Item')

    dataset = create_one_hot(dataset, 'Store_Type', 'Store_Type')
    dataset = create_one_hot(dataset, 'Store_Location_Type', 'Store_Location')
    dataset = create_one_hot(dataset, 'Store_Start_Year', '')
    return dataset

data = prepare_dataset(data)
data.head()

,Item_Price,Item_Store_Returns,Log_Item_Price,Sqrt_Item_Price,Reciprocal_Item_Price,Item_Price_Inv,Cubert_Item_Price,Item_Baking Goods,Item_Breads,Item_Breakfast,...,Store_Location_Cluster 3,_1992,_1994,_2004,_2005,_2006,_2009,_2011,_2014,_2016
Item_Store_ID,,,,,,,,,,,,,,,,,,,,,
DRA12_BABATUNJI010,357.54,709.08,5.879247,18.908728,0.002797,0.002797,7.097546,False,False,False,...,True,False,False,False,True,False,False,False,False,False
DRA12_BABATUNJI013,355.79,6381.69,5.874341,18.862396,0.002811,0.002811,7.085947,False,False,False,...,True,False,True,False,False,False,False,False,False,False
DRA12_BABATUNJI017,350.79,6381.69,5.860188,18.729389,0.002851,0.002851,7.052597,False,False,False,...,False,False,False,False,False,False,False,False,True,False
DRA12_BABATUNJI018,355.04,2127.23,5.872230,18.842505,0.002817,0.002817,7.080965,False,False,False,...,True,False,False,False,False,False,False,False,False,True
DRA12_BABATUNJI035,354.79,2481.77,5.871526,18.835870,0.002819,0.002819,7.079302,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [8]:
# Check for missing values.
data.isnull().sum() 

Item_Price                      0
Item_Store_Returns              0
Log_Item_Price                  0
Sqrt_Item_Price                 0
Reciprocal_Item_Price           0
Item_Price_Inv                  0
Cubert_Item_Price               0
Item_Baking Goods               0
Item_Breads                     0
Item_Breakfast                  0
Item_Canned                     0
Item_Dairy                      0
Item_Frozen Foods               0
Item_Fruits and Vegetables      0
Item_Hard Drinks                0
Item_Health and Hygiene         0
Item_Household                  0
Item_Meat                       0
Item_Others                     0
Item_Seafood                    0
Item_Snack Foods                0
Item_Soft Drinks                0
Item_Starchy Foods              0
Store_Type_Grocery Store        0
Store_Type_Supermarket Type1    0
Store_Type_Supermarket Type2    0
Store_Type_Supermarket Type3    0
Store_Location_Cluster 1        0
Store_Location_Cluster 2        0
Store_Location

In [9]:
# Create a pipeline for the model.
degree = 2
pipeline = Pipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
        ('scaler', StandardScaler()),
        ('regressor', Lasso(alpha=60.5, random_state=42))
    ])

X = data.drop(columns='Item_Store_Returns').values
y = data['Item_Store_Returns'].values

pipeline.fit(X, y)

Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),
                ('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=60.5, random_state=42))])

In [10]:
# Total Training loss                
y_pred = pipeline.predict(X)
train_loss = root_mean_squared_error(y, y_pred)
print(train_loss) 

2894.191031314678


In [11]:
# READ-IN test set
test_data = pd.read_csv('test.csv', index_col=2)
test_data.head()

,Item_ID,Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,Store_Type
Item_Store_ID,,,,,,,,,,,
DRA59_BABATUNJI010,DRA59,BABATUNJI010,8.270,Normal Sugar,0.214125,Soft Drinks,459.98,2005,NaN,Cluster 3,Grocery Store
DRA59_BABATUNJI013,DRA59,BABATUNJI013,8.270,Normal Sugar,0.127821,Soft Drinks,464.98,1994,High,Cluster 3,Supermarket Type1
DRB01_BABATUNJI013,DRB01,BABATUNJI013,7.390,Low Sugar,0.082171,Soft Drinks,477.38,1994,High,Cluster 3,Supermarket Type1
DRB13_BABATUNJI010,DRB13,BABATUNJI010,6.115,Normal Sugar,0.011791,Soft Drinks,472.63,2005,NaN,Cluster 3,Grocery Store
DRB13_BABATUNJI013,DRB13,BABATUNJI013,6.115,Normal Sugar,0.007038,Soft Drinks,473.13,1994,High,Cluster 3,Supermarket Type1


In [12]:
# test_data.info()

In [13]:
# test_data.isnull().sum()

In [14]:
test_data = prepare_dataset(test_data)

In [15]:
test_data.head()

,Item_Price,Log_Item_Price,Sqrt_Item_Price,Reciprocal_Item_Price,Item_Price_Inv,Cubert_Item_Price,Item_Baking Goods,Item_Breads,Item_Breakfast,Item_Canned,...,Store_Location_Cluster 3,_1992,_1994,_2004,_2005,_2006,_2009,_2011,_2014,_2016
Item_Store_ID,,,,,,,,,,,,,,,,,,,,,
DRA59_BABATUNJI010,459.98,6.131183,21.447144,0.002174,0.002174,7.719331,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
DRA59_BABATUNJI013,464.98,6.141994,21.563395,0.002151,0.002151,7.747200,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
DRB01_BABATUNJI013,477.38,6.168313,21.849027,0.002095,0.002095,7.815464,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
DRB13_BABATUNJI010,472.63,6.158313,21.740055,0.002116,0.002116,7.789455,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
DRB13_BABATUNJI013,473.13,6.159370,21.751552,0.002114,0.002114,7.792201,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False


In [16]:
# pdmodel.predict(test_data)
# test_data.drop(columns=['Item_Type', 'Item_ID'], inplace=True)
submission = pd.DataFrame({'Item_Store_Returns': pipeline.predict(test_data.values)}, test_data.index)
submission.to_csv('submission2.csv')

In [17]:
pipeline.predict(test_data.values)

array([ 1271.38067489,  7556.75749275,  7757.65136042, ...,
       10010.71938746,  1303.47556544,  7582.19723575], shape=(3532,))